In [1]:
from weavepy import *
from config import BDD_VERSION
sns.set_context("talk")

In [2]:
# --- Parameters for loading data ---
# It is expected that these will be changed rarely
## Variables
climate_variable = "TA"
energy_variable = "SPV"
vars = [climate_variable, energy_variable]
techno = "60" # Techno needs to be specified for energy variables
## Temporal aggregation
aggregation_frequency = "D"
aggregation_function = "mean"
## Selection
country = "FR"
models = []
scenarios = []

In [3]:
data = load_vars([climate_variable, energy_variable], bdd_version = BDD_VERSION,
              countries = [country], technos = ["NA", techno], models = models, scenarios = scenarios, 
              aggregation_frequency = aggregation_frequency, aggregation_function = aggregation_function,
              verbose=True)

Loading country n°1/1 (FR)
|_Variable n°1/2 (TA)
|__Techno n°1/1 (NA)
|__ > File already exists, loading from cache: /ec/res4/scratch/gbr6848/PECD4.2/TA_NA_D-mean_FR_PECD4.2.nc
|_Variable n°2/2 (SPV)
|__Techno n°1/1 (60)
|__ > File already exists, loading from cache: /ec/res4/scratch/gbr6848/PECD4.2/SPV_60_D-mean_FR_PECD4.2.nc


In [ ]:
# --- Period ---
# These can be changed and adjusted easily
historical_period = (1990, 2020)
future_period = (2070, 2100)

In [ ]:
# --- Thresholds ---
# These can be changed and adjusted easily
comparisons = { # For each variable, choose among '<', '<=', '>', or '>='
    climate_variable:'<=', 
    energy_variable:'<',
}
thresholds = { # Number in units of variable
    climate_variable: 5,
    energy_variable: 0.05
}

In [ ]:
# Identify problematic days
days = {}
for var in vars:
    days[var] = identify_pb_days(data[var], comparisons[var], thresholds[var])
days["compound"] = xr.concat(list(days.values()), dim = "var").all(dim = "var").rename("compound")
# Identify events
events = {}
for var in vars + ["compound"]:
    events[var] = identify_events_whole_base(days[var])

100%|██████████████████████████████████████████| 35/35 [00:00<00:00, 266.42it/s]


## Number of events

In [ ]:
#event_count_barplot_multi(list(events.values()), historical_period, future_period)

In [8]:
#from weavepy.plot_time_series import nb_event_timeseries
#import plotly.graph_objects as go
#go.FigureWidget(nb_event_timeseries(events["TA"]))

In [9]:
nb_event_timeseries_multi(list(events.values()))

FigureWidget({
    'data': [{'customdata': [['SP126', 'AWI-_AWCM', 2015, 2035], ['SP126', 'AWI-
                             _AWCM', 2016, 2036], ['SP126', 'AWI-_AWCM', 2017,
                             2037], ['SP126', 'AWI-_AWCM', 2018, 2038], ['SP126',
                             'AWI-_AWCM', 2019, 2039], ['SP126', 'AWI-_AWCM', 2020,
                             2040], ['SP126', 'AWI-_AWCM', 2021, 2041], ['SP126',
                             'AWI-_AWCM', 2022, 2042], ['SP126', 'AWI-_AWCM', 2023,
                             2043], ['SP126', 'AWI-_AWCM', 2024, 2044], ['SP126',
                             'AWI-_AWCM', 2025, 2045], ['SP126', 'AWI-_AWCM', 2026,
                             2046], ['SP126', 'AWI-_AWCM', 2027, 2047], ['SP126',
                             'AWI-_AWCM', 2028, 2048], ['SP126', 'AWI-_AWCM', 2029,
                             2049], ['SP126', 'AWI-_AWCM', 2030, 2050], ['SP126',
                             'AWI-_AWCM', 2031, 2051], ['SP126', 'AWI-_AWCM', 2

## Duration 

In [9]:
event_duration_hist_multi(list(events.values()), historical_period, future_period)

FigureWidget({
    'data': [{'fill': 'tozeroy',
              'fillcolor': 'rgba(23,60,102,0.15)',
              'hovertemplate': '%{x}<br>proportion: %{y:.3f}<extra></extra>',
              'legendgroup': np.str_('SP126'),
              'line': {'color': 'rgb(23,60,102)', 'width': 3},
              'mode': 'lines',
              'name': np.str_('SP126'),
              'showlegend': True,
              'type': 'scatter',
              'uid': '95980162-cfce-4ab2-b9b9-33166e3b0961',
              'x': {'bdata': ('AAAAAAAA4D8AAAAAAAD4PwAAAAAAAP' ... 'AAAIA8QAAAAAAAgDxAAAAAAACAPUA='),
                    'dtype': 'f8'},
              'xaxis': 'x',
              'y': {'bdata': ('uKmCkBk40j+4qYKQGTjSP7BBl3YSDM' ... 'AAAAAAAAN4TrNXuTY/A3hOs1e5Nj8='),
                    'dtype': 'f8'},
              'yaxis': 'y'},
             {'legendgroup': np.str_('SP126'),
              'line': {'color': 'rgb(23,60,102)', 'dash': 'dash', 'width': 2},
              'mode': 'lines',
              'name': 'S

## Seasonality

In [10]:
event_seasonality_kde_multi(list(days.values()), historical_period, future_period)

FigureWidget({
    'data': [{'customdata': {'bdata': ('AAAAAADQZsAUqPUrBVpmwClQ61cK5G' ... 'tXCuRlQBWo9SsFWmZAAAAAAADQZkA='),
                             'dtype': 'f8'},
              'hovertemplate': 'DOY: %{customdata:.0f}<br>Density: %{r:.3f}<extra></extra>',
              'legendgroup': np.str_('SP126'),
              'line': {'color': 'rgb(23,60,102)'},
              'mode': 'lines',
              'name': np.str_('SP126'),
              'r': {'bdata': ('86JDQ7VfVTBsi6ez8x5PLxyJFt30W0' ... '0OuSFgMujLxu9ZrFsxEaNDQ7VfVTA='),
                    'dtype': 'f8'},
              'showlegend': True,
              'subplot': 'polar',
              'theta': {'bdata': ('AAAAAACAZsCLLrroogtmwBdddNFFl2' ... 'TRRZdlQIwuuuiiC2ZAAAAAAACAZkA='),
                        'dtype': 'f8'},
              'type': 'scatterpolar',
              'uid': '51d02b88-31e4-468b-80a5-bceb71303ca3'},
             {'customdata': {'bdata': ('AAAAAADQZsAUqPUrBVpmwClQ61cK5G' ... 'tXCuRlQBWo9SsFWmZAAAAAAADQZkA='),
    